In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# default_exp submissionshandler
# all_local

In [ ]:
# export
import pickle
import json
from collections import Counter
from pprint import pprint
import sqlite3
from pathlib import Path
from steroidsornot.database import Session
from steroidsornot.submission import Submission

In [ ]:
from fastcore.test import *

## Unpickling, cleaning downloaded pushshift data

In [ ]:
# export
class SubmissionsHandler():
    def __init__(self, path, remove_irrelevant=True):
        self.path = path
        self.submissions = []
        
        self._unpickle(path, remove_irrelevant)
        
    def export_to_database(self):
        session = Session()
        
        for pushshift_submission in self.submissions:
            database_submission = Submission(
                data=json.dumps(pushshift_submission)
            )

            session.add(database_submission)

        session.commit()
        session.close()
    
    def common_domains(self):
        '''
            Show the number of posts that link to each domain
        '''
        pprint(Counter([post['domain'] for post in self.submissions]).most_common())

    def _unpickle(self, path, remove_irrelevant):
        self.submissions = []
        with open(path, "rb") as file:
            while 1:
                try:
                    self.submissions += pickle.load(file)
                except EOFError:
                    break
            
        print(f'Unpickled {len(self.submissions)} objects.')
        
        if remove_irrelevant:
            self._select_mostly_useful()
            
    def is_useful(self, post):
        if (post['num_comments'] >= 2 and
                    'selftext' in post and
                    post['selftext'] != '[deleted]' and
                    post['selftext'] != '[removed]' and
                    post.get('removed_by_category') == None and
                    post.get('link_flair_text') != 'Meme' and
                    post.get('domain') == 'i.redd.it'):
            return True
        else:
            return False
    
    def _select_mostly_useful(self):
        '''
        Remove mostly invalid posts, stuff like:
        few comments (so no label),
        no selftext are often not useful
        post was deleted or removed

        It's not a complete solution, as the pushshift data
        doesn't get updated when the post changes, so there
        are many posts which were deleted after fact.
        '''
        mostly_useful = []
        for post in self.submissions:
            if self.is_useful(post):
                mostly_useful.append(post)
        
        self.submissions = mostly_useful
        
        print(f'{len(self.submissions)} are mostly useful.\n')
        
        # Show that no posts were deleted when pushshift downloaded them
        selftext = [post['selftext'] for post in self.submissions]
        print('This should show no [deleted] or [removed] entries:')
        print(Counter(selftext).most_common())

In [ ]:
submission_handler = SubmissionsHandler('data/all_nattyorjuice_submissions.pkl')

Unpickled 41725 objects.
16260 are mostly useful.

This should show no [deleted] or [removed] entries:
[('', 16260)]


In [ ]:
test_eq(len(submission_handler.submissions), 16260)